In [1]:
import pandas as pd
import numpy as np
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('./billboard_lyrics_1964-2015.csv', encoding='ISO-8859-1')

df.dropna(axis=0, how='any', subset=['Lyrics'], inplace=True)

df['Lyrics'].str.replace(r'[^a-zA-Z ]', '')

stopwords = nltk.corpus.stopwords.words('english')

In [1]:
stopwords.extend(add_stopwords)

NameError: name 'stopwords' is not defined

In [5]:
sample = df[(df['Year'] <= 2010) & (df['Year'] > 2005)]

sample['ID'] = sample['Song'] + ' - ' +sample['Artist']

# sample.drop(columns=['Rank', 'Year', 'Source', 'Song', 'Artist'], axis=1, inplace=True)

/home/danish/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
vectorizer = CountVectorizer(stop_words=stopwords)
doc_word = vectorizer.fit_transform(sample['Lyrics'])

doc_word.shape

(484, 10834)

In [7]:
pd.DataFrame(doc_word.toarray(), index=sample['ID'], columns=vectorizer.get_feature_names()).head(1)

,07,10,100,125,12yearold,13,13th,15,15th,16,...,zeros,zip,zippers,zombie,zone,zonecheck,zonenow,zonin,zoom,zucchinis
ID,,,,,,,,,,,,,,,,,,,,,
bad day - daniel powter,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
lsa = TruncatedSVD(4)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.0169309 , 0.02691742, 0.0250065 , 0.02235126])

In [9]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ['C1', 'C2', 'C3', 'C4'],
             columns = vectorizer.get_feature_names())

In [10]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [11]:
display_topics(lsa, vectorizer.get_feature_names(), 15)


Topic  0
love, know, baby, girl, one, right, take, way, time, back, could, never, need, life, tell

Topic  1
bay, know, rock, club, low, right, ridin, lean, party, wee, star, body, bayay, hollerin, tryin

Topic  2
bay, love, one, club, game, away, bayay, hollerin, play, lockdown, runaway, help, baby, drug, dougie

Topic  3
rock, love, party, star, boom, lean, body, bay, hips, right, game, totally, bed, dance, shoulder


In [3]:
add_stopwords = ['la', 'ay', 'oh', 'ohh', 'ooh', 'yeah', 'hey', 'im', 'like', 'get', 'go', 'na', 'got', 'duh', 
                 'ya', 'ah', 'let', 'cuz', 'wit', 'da', 'gonna', 'cause', 'imma', 'dem', 'dey', 'dats', 'wat',
                 'dont', 'cant', 'wanna', 'see', 'make', 'want', 'youre', 'keep', 'lets', 'dat', 'ba', 'bah', 'aint',
                 'aah', 'say', 'aye', 'come'
                ]